In [107]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got
import matplotlib.pyplot as plt

In [108]:
sz_file_path = "./dump.csv"
sz_file_csv = pd.read_csv(sz_file_path,usecols=['initiatingDeviceName','receivingDeviceName', 'connectionCount'],
                          header=0,
                       
                        )

In [109]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "from",
        "receivingDeviceName": "to",
        "connectionCount": "weight"
    }
)

In [110]:
df_grouped = df.groupby(['from','to'], as_index=False)['weight'].sum()

In [111]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "id",
        "receivingDeviceName": "to",
        "connectionCount": "weight",

    }
)
df_nodes = df.groupby(['id'], as_index=False)['weight'].sum()

In [114]:

Jaal(df_grouped,df_nodes).plot(vis_opts={'height': '600px', # change height
#                                     'interaction':{'hover': True}, # turn on-off the hover                                    
                                    'physics':{'enabled': False}})                     # Estatico





Parsing the data...Done


No trigger
